In [ ]:
import numpy as np
import pandas as pd
import itertools
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
import re 
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import plotly
from sklearn.preprocessing import StandardScaler, scale
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns

## Подготовка данных

In [ ]:
tab_2014_1 = pd.read_excel('Macro data.xlsx', sheet_name = 'Сырые данные')
tab_2014_ = pd.read_excel('Macro data.xlsx', sheet_name = 'Данные')

print(tab_2014_.head(3))
print(tab_2014_.tail(3))
scaler = MinMaxScaler(feature_range=(0, 100))
scaler = scaler.fit(tab_2014_.iloc[:,1:])
try1 = scaler.transform(tab_2014_.iloc[:,1:])
tab_2014__ = pd.DataFrame(try1, columns=tab_2014_.iloc[:,1:].columns)

var_name = 'nonfood_goods' ## определяем целевую переменную

tab_2014 = tab_2014__.filter(['ipd', var_name, 'oil', 'real_rate', 'vix'], axis = 1)
for i in tab_2014.columns: 
    tab_2014[i+'-1'] = tab_2014[i].shift().reset_index(drop = True)
    tab_2014[i+'-2'] = tab_2014[i+'-1'].shift().reset_index(drop = True)
    tab_2014[i+'-3'] = tab_2014[i+'-2'].shift().reset_index(drop = True) 

In [ ]:
tab_2014_12 = tab_2014_1.filter(['Продовольственные', 'Непродовольственные', 'Российский индекс волатильности', 'Цена на нефть (BRENT)', 'Реальный валютный курс', 'Индекс потребительского доверия'])[:-4]
plotly.offline.init_notebook_mode()

for i in range(len(tab_2014_12.columns)):
    data=(plotly.graph_objs.Scatter(x=tab_2014_1.Дата, 
                                      y=pd.DataFrame(tab_2014_12).iloc[:,i], 
                                      mode='lines'))

    layout = go.Layout(legend=dict(orientation="h"),font=dict(size=15,),autosize=False, width=1000,height=700,
        paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', xaxis = dict(showticklabels = True),
        yaxis = dict(showgrid = True, gridcolor = '#e3e3e3', gridwidth = 1, zerolinecolor = '#969696', zerolinewidth = 2,
        linecolor = '#636363', linewidth = 1, showticklabels = True))

    fig = go.Figure(data=data, layout = layout)

    plotly.offline.iplot(fig) 

In [ ]:
prod = pd.read_excel('prod.xlsx')
prod.index = prod['Date']
prod = prod.drop(['Date'], axis = 1)
print(prod.tail(3))

prod_detrend = np.log(prod).diff(12)
X1 = prod_detrend[prod_detrend.index>='2015-01-01']

scaler = MinMaxScaler(feature_range=(0, 100))
scaler = scaler.fit(X1)
try1 = scaler.transform(X1)
prod_detrend = pd.DataFrame(try1, columns=prod.columns)
pd.DataFrame(prod).describe().round(2)

In [ ]:
plotly.offline.init_notebook_mode()
data = []
for i in range(len(prod.columns)):
    data.append(plotly.graph_objs.Scatter(x=prod.index[132:], 
                                      y=prod.iloc[:,i][132:], 
                                      mode='lines', name = prod.columns[i]))
    
layout = go.Layout(font=dict(size=18,),autosize=False, width=1000,height=700,
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', xaxis = dict(showticklabels = True),
    yaxis = dict(showgrid = True, gridcolor = '#e3e3e3', gridwidth = 1, zerolinecolor = '#969696', zerolinewidth = 2,
    linecolor = '#636363', linewidth = 1, showticklabels = True))

fig = go.Figure(data=data, layout = layout)

plotly.offline.iplot(fig)  

In [ ]:
neprod = pd.read_excel('neprod.xlsx')
neprod.index = neprod['Date']
neprod = neprod.drop(['Date'], axis = 1)
neprod.head(3)

neprod_detrend = np.log(neprod).diff(12)
X1 = neprod_detrend[neprod_detrend.index>='2015-01-01']

scaler = MinMaxScaler(feature_range=(0, 100))
scaler = scaler.fit(X1)
try1 = scaler.transform(X1)
neprod_detrend = pd.DataFrame(try1, columns=neprod.columns)
pd.DataFrame(neprod_detrend).describe().round(2)

In [ ]:
plotly.offline.init_notebook_mode()
data = []
for i in range(len(neprod.columns)):
    data.append(plotly.graph_objs.Scatter(x=neprod.index[132:], 
                                      y=pd.DataFrame(neprod).iloc[:,i][132:], 
                                      mode='lines', name = neprod.columns[i]))

layout = go.Layout(font=dict(size=18,),autosize=False, width=1000,height=700,
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', xaxis = dict(showticklabels = True),
    yaxis = dict(showgrid = True, gridcolor = '#e3e3e3', gridwidth = 1, zerolinecolor = '#969696', zerolinewidth = 2,
    linecolor = '#636363', linewidth = 1, showticklabels = True))

fig = go.Figure(data=data, layout = layout)

plotly.offline.iplot(fig)  

# Прогнозы

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs(y_true - y_pred))

In [ ]:
variables = [tab_2014['ipd'], tab_2014['ipd-1'], tab_2014['ipd-2'], tab_2014['ipd-3'], 
             tab_2014['oil'], tab_2014['oil-1'], tab_2014['oil-2'],tab_2014['oil-3'],
             tab_2014['vix'], tab_2014['vix-1'], tab_2014['vix-2'],tab_2014['vix-3'],
             tab_2014['real_rate'], tab_2014['real_rate-1'],tab_2014['real_rate-2'],tab_2014['real_rate-3'],
             tab_2014[var_name+'-1'], tab_2014[var_name+'-2'], tab_2014[var_name+'-3']]

In [ ]:
tab_2014.tail(2)

## На несколько шагов вперед на фиксированной обучающей выборке

In [ ]:
combs = []
for i in range(1, 4):
    els = [list(x) for x in itertools.combinations(variables, i)]
    combs.extend(els)
len(combs)

In [ ]:
def match_func(index, lag, results):
    pattern = re.compile(var_name + index)
    match  = pattern.search(str(combs[g]))
    if match:
        start = len(results.fittedvalues)
        predictions = []
        for i in range(start, len(y)):
            ### Назначаем текущее значение прогноза у его лагу.  
            X[var_name+index][i-lag+1] = results.predict(X[i:i+1])

            ## добавляем прогнозное наблюдение
            predictions.append(X[var_name+index][i-lag+1])
    return predictions

def result(X, y):
    t = X.isnull().sum(axis = 0).max()
    y = y[t:]            
    X = X[t:] 
    X = X.reset_index(drop = True)
    y = y.reset_index(drop = True)
    X = sm.add_constant(X)
    n = round(len(X)*0.86)
    return X, y, n, t

def get_predictions():
    try:
        predictions = match_func('-1', 1, results = results)
    except:
        try:
            predictions = match_func('-2', 2, results = results)
        except:
            try:
                predictions = match_func('-3', 3, results = results)
            except:
                predictions = results.predict(X[n:])
    return predictions

In [ ]:
### Составляем базовые модели 

inf = []
for m in range(len(combs)):
    X = pd.DataFrame(combs[m]).T
    y = tab_2014.filter([var_name])
    X, y, n, t = result(X, y)
    model = sm.OLS(y[:n], X[:n])
    results = model.fit(cov_type='HC3')
    predictions = get_predictions()
    columns = X.columns
    MAE = mean_absolute_percentage_error(y[n:][t:][var_name], predictions)
    p = results.pvalues
    k=[]
    for i in range(0,len(p)):
        if p[i] < 0.05:
            k.append(p.index[i])
            
    r = results.rsquared_adj
    inf.append([list(X.columns), k, r, MAE])

ols_all = pd.DataFrame(inf, columns = ['Регрессоры', 'Значимые регрессоры', 'Скорректированный R^2', 'MAE'])
p_values = []
regressors = []
for i in range(len(ols_all)):
    p_values.append(", ".join(ols_all['Значимые регрессоры'][i])) 
    regressors.append(", ".join(ols_all['Регрессоры'][i])) 
ols_all['Значимые регрессоры'] = p_values
ols_all['Регрессоры'] = regressors

df = pd.DataFrame()
for j in range(len(ols_all['Значимые регрессоры'])):
    if ', '.join(ols_all['Регрессоры'][j].split(', ')[1:]) in ols_all['Значимые регрессоры'][j]:
        df = df.append(ols_all[ols_all.index == j])
models = df.sort_values('Скорректированный R^2', ascending=False).reset_index(drop=True)[:10]

In [ ]:
goods = neprod_detrend ## Выбираем тренды к целевой переменной
combs = []
for i in range(1, 3):
    els = [list(x) for x in itertools.combinations(goods, i)]
    combs.extend(els)

In [ ]:
s = pd.DataFrame()
for g in range(len(combs)):
    inf = []
    for m in range(len(models)):
        splited = models['Регрессоры'][m].split(', ')
        try:
            X1 = tab_2014.filter([splited[1], splited[2], splited[3]])
        except:
            try:
                X1 = tab_2014.filter([splited[1], splited[2]])
            except:
                X1 = tab_2014.filter([splited[1]])   
        X = X1.join(goods[combs[g]], how = 'left')
        y = tab_2014.filter([var_name])
        X, y, n, t = result(X, y)
        model = sm.OLS(y[:n], X[:n])
        results = model.fit(cov_type='HC3')
        predictions = get_predictions()
        columns = X.columns
        MAE = mean_absolute_percentage_error(y[n:][t:][var_name], predictions)
        p = results.pvalues
        k=[]
        for i in range(0,len(p)):
            if p[i] < 0.05:
                k.append(p.index[i])

        r = results.rsquared_adj
        inf.append([list(X.columns), k, r, MAE])
    ols_all = pd.DataFrame(inf, columns = ['Регрессоры', 'Значимые регрессоры', 'Скорректированный R^2', 'MAE'])
    table = models.join(ols_all, how = 'left', rsuffix=' (тренды)')
    table['Разница MAE'] = table['MAE (тренды)'] - table['MAE']
    table['Разница R^2'] = table['Скорректированный R^2 (тренды)'] - table['Скорректированный R^2']
    table1 = table[(table['Разница MAE']<0) & (table['Разница R^2']>0)].reset_index(drop = True)
    for j in range(len(table1['Значимые регрессоры (тренды)'])):
        if combs[g][0] in table1['Значимые регрессоры (тренды)'][j]:
            s = s.append(table1[table1.index == j])

In [ ]:
s_mae = s.sort_values(by='Разница MAE', ascending=True).reset_index(drop = True)[:10]

In [ ]:
len(s)

In [ ]:
predictions_total_bench = []
predictions_total_gt = []
for i in range(len(s_mae)):
    X = tab_2014.filter(s_mae.iloc[:,0][i].split())
    y = tab_2014.filter([var_name])
    X, y, n, t = result(X,y)
    model = sm.OLS(y[:n], X[:n])
    results = model.fit(cov_type='HC3')
    predictions = get_predictions()
    predictions_total_bench.append(predictions)
    X = tab_2014.filter(s_mae.iloc[:,0][i].split()).join(goods[s_mae.iloc[:,4][i][len(s_mae.iloc[:,0][i].split()):]], how = 'left')
    y = tab_2014.filter([var_name])
    X, y, n, t = result(X,y)
    model = sm.OLS(y[:n], X[:n])
    results = model.fit(cov_type='HC3')
    predictions1 = get_predictions()
    predictions_total_gt.append(predictions1)

mean_gt = pd.DataFrame(predictions_total_gt).mean(axis = 0)
mean_bch = pd.DataFrame(predictions_total_bench).mean(axis = 0)

plotly.offline.init_notebook_mode()

data = [plotly.graph_objs.Scatter(x=prod[132+t+n:].index, 
                                  y=y.iloc[:,0][n:], mode='lines', name = 'Непродовольственный ретейл'),
       plotly.graph_objs.Scatter(x=prod[132+n+t:].index, 
                                  y=mean_bch[:], mode='lines', name = 'Усредненные прогнозы базовых моделей'),
       plotly.graph_objs.Scatter(x=prod[132+n+t:].index, 
                                 y=mean_gt[:], mode='lines', name = 'Усредненные прогнозы расширенных моделей')]

layout = go.Layout(legend=dict(orientation="h"),font=dict(size=15,),autosize=False, width=1000,height=700,
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', xaxis = dict(showticklabels = True),
    yaxis = dict(showgrid = True, gridcolor = '#e3e3e3', gridwidth = 1, zerolinecolor = '#969696', zerolinewidth = 2,
    linecolor = '#636363', linewidth = 1, showticklabels = True))

fig = go.Figure(data=data, layout = layout)

plotly.offline.iplot(fig)  

In [ ]:
s_temp = s.sort_values(by='Разница R^2', ascending=False).reset_index(drop = True)
for i in neprod.columns:
    s1 = pd.DataFrame()
    for j in range(len(s_temp['Регрессоры (тренды)'])):
        if i in ' '.join(s_temp['Регрессоры (тренды)'][j]): 
            s1 = s1.append(s_temp[s_temp.index == j])
    print(len(s1))

## На один шаг вперед на растущей обучающей выборке

In [ ]:
import itertools
combs = []
for i in range(1, 4):
    els = [list(x) for x in itertools.combinations(variables, i)]
    combs.extend(els)
len(combs)

In [ ]:
inf = []
for m in range(len(combs)):
    X = pd.DataFrame(combs[m]).T
    y = tab_2014.filter([var_name])
    X, y, n, t = result(X, y)
    model = sm.OLS(y[:n], X[:n])
    results = model.fit(cov_type='HC3')
    
    start = results.fittedvalues.index[-1]
    predictions = []
    for i in range(start, y.index[-1]):
        ## добавляем прогнозное наблюдение
        a = results.predict(X[i:i+1])
        predictions.append(a[i])
        model = sm.OLS(y[:i], X[:i])
        results = model.fit(cov_type='HC3')

    columns = X.columns
    r = results.rsquared_adj
    p = results.pvalues
    k=[]
    for i in range(0,len(p)):
        if p[i] < 0.05:
            k.append(p.index[i])
    MAE = mean_absolute_percentage_error(y[n:][var_name], predictions)         
    inf.append([list(X.columns), k, r, MAE])
        
ols_all = pd.DataFrame(inf, columns = ['Регрессоры', 'Значимые регрессоры', 'Скорректированный R^2', 'MAE'])
p_values = []
regressors = []
for i in range(len(ols_all)):
    p_values.append(", ".join(ols_all['Значимые регрессоры'][i])) 
    regressors.append(", ".join(ols_all['Регрессоры'][i])) 
ols_all['Значимые регрессоры'] = p_values
ols_all['Регрессоры'] = regressors

df = pd.DataFrame()
for j in range(len(ols_all['Значимые регрессоры'])):
    if ', '.join(ols_all['Регрессоры'][j].split(', ')[1:]) in ols_all['Значимые регрессоры'][j]:
        df = df.append(ols_all[ols_all.index == j])
models = df.sort_values('Скорректированный R^2', ascending=False).reset_index(drop=True)[:10]

In [ ]:
goods = neprod_detrend ## Выбираем тренды к целевой переменной
combs = []
for i in range(1, 3):
    els = [list(x) for x in itertools.combinations(goods, i)]
    combs.extend(els)

In [ ]:
s = pd.DataFrame()
for g in range(len(combs)):
    inf = []
    for m in range(len(models)):
        splited = models['Регрессоры'][m].split(', ')
        try:
            X1 = tab_2014.filter([splited[1], splited[2], splited[3]])
        except:
            try:
                X1 = tab_2014.filter([splited[1], splited[2]])
            except:
                X1 = tab_2014.filter([splited[1]])   
        X = X1.join(goods[combs[g]], how = 'left')
        y = tab_2014.filter([var_name])
        X, y, n, t = result(X, y)
        model = sm.OLS(y[:n], X[:n])
        results = model.fit(cov_type='HC3')
        start = results.fittedvalues.index[-1]
        predictions = []
        for i in range(start, y.index[-1]):
            ## добавляем прогнозное наблюдение
            a = results.predict(X[i:i+1])
            predictions.append(a[i])
            model = sm.OLS(y[:i], X[:i])
            results = model.fit(cov_type='HC3')
        columns = X.columns
        p = results.pvalues
        k=[]
        for i in range(0,len(p)):
            if p[i] < 0.05:
                k.append(p.index[i])
        r = results.rsquared_adj
        MAE = mean_absolute_percentage_error(y[n:][var_name], predictions)
        inf.append([list(X.columns), k, r, MAE])    
    ols_all = pd.DataFrame(inf, columns = ['Регрессоры', 'Значимые регрессоры', 'Скорректированный R^2', 'MAE'])
    table = models.join(ols_all, how = 'left', rsuffix=' (тренды)')
    table['Разница MAE'] = table['MAE (тренды)'] - table['MAE']
    table['Разница R^2'] = table['Скорректированный R^2 (тренды)'] - table['Скорректированный R^2']
    table1 = table[(table['Разница MAE']<0) & (table['Разница R^2']>0)].reset_index(drop = True)
    for j in range(len(table1['Значимые регрессоры (тренды)'])):
        if combs[g][0] in table1['Значимые регрессоры (тренды)'][j]:
            s = s.append(table1[table1.index == j])

In [ ]:
s_r2_1 = s.sort_values(by='Разница R^2', ascending=False).reset_index(drop = True)[:10]
s_mae_1 = s.sort_values(by='Разница MAE', ascending=True).reset_index(drop = True)[:10]

In [ ]:
len(s)

In [ ]:
predictions_total_bench = []
predictions_total_gt = []

r_bench = []
r_gt = []

variable = s_mae_1

for i in range(len(variable)):
    X = tab_2014.filter(variable.iloc[:,0][i].split())
    y = tab_2014.filter([var_name])
    X, y, n, t = result(X, y)
    model = sm.OLS(y[:n], X[:n])
    results = model.fit(cov_type='HC3')
    predictions = []
    r_sq = []
    start = results.fittedvalues.index[-1]
    for k in range(start, y.index[-1]):
        ## добавляем прогнозное наблюдение
        a = results.predict(X[k:k+1])
        predictions.append(a[k])
        model = sm.OLS(y[:k], X[:k])
        results = model.fit(cov_type='HC3')
        r = results.rsquared_adj
        r_sq.append(r)
    
    predictions_total_bench.append(predictions)
    r_bench.append(r_sq)
    
    X = tab_2014.filter(variable.iloc[:,0][i].split()).join(goods[variable.iloc[:,4][i][len(variable.iloc[:,0][i].split()):]], how = 'left')
    y = tab_2014.filter([var_name])
    X, y, n, t = result(X, y)
    model = sm.OLS(y[:n], X[:n])
    results = model.fit(cov_type='HC3')
    start = results.fittedvalues.index[-1]
    predictions1 = []
    r_sq1 = []
    for k in range(start, y.index[-1]):
        ## добавляем прогнозное наблюдение
        a = results.predict(X[k:k+1])
        predictions1.append(a[k])
        model = sm.OLS(y[:k], X[:k])
        results = model.fit(cov_type='HC3')
        r = results.rsquared_adj
        r_sq1.append(r)
        
    predictions_total_gt.append(predictions1)
    r_gt.append(r_sq1)
    
mean_gt = pd.DataFrame(predictions_total_gt).mean(axis = 0)
mean_bch = pd.DataFrame(predictions_total_bench).mean(axis = 0)

mean_r_bench = pd.DataFrame(r_bench).mean(axis = 0)
mean_r_gt = pd.DataFrame(r_gt).mean(axis = 0)

plotly.offline.init_notebook_mode()

for i in range(len(variable.MAE)):

    if variable.MAE[i] == s_mae_1.MAE[i]:
        data = [plotly.graph_objs.Scatter(x=prod[132+n+t:].index, 
                                          y=y.iloc[:,0][n:], mode='lines', name = 'Непродовольственный ретейл'),
               plotly.graph_objs.Scatter(x=prod[132+n+t:].index, 
                                          y=mean_bch, mode='lines', name = 'Усредненные прогнозы базовых моделей'),
               plotly.graph_objs.Scatter(x=prod[132+n+t:].index, 
                                         y=mean_gt, mode='lines', name = 'Усредненные прогнозы расширенных моделей')]
    else:
        data = [plotly.graph_objs.Scatter(x=prod[132+n+t:].index, 
                                          y=mean_r_bench, mode='lines', name = 'Усредненный коэффициент детерминации базовых моделей', line=dict(color="red")),
               plotly.graph_objs.Scatter(x=prod[132+n+t:].index, 
                                         y=mean_r_gt, mode='lines', name = 'Усредненный коэффициент детерминации расширенных моделей', line=dict(color="#10CDAC"))]


layout = go.Layout(legend=dict(orientation="h"),font=dict(size=15,),autosize=False, width=1000,height=700,
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', xaxis = dict(showticklabels = True),
    yaxis = dict(showgrid = True, gridcolor = '#e3e3e3', gridwidth = 1, zerolinecolor = '#969696', zerolinewidth = 2,
    linecolor = '#636363', linewidth = 1, showticklabels = True))

fig = go.Figure(data=data, layout = layout)

plotly.offline.iplot(fig)  

In [ ]:
ss = s.sort_values(by='Разница R^2', ascending=False).reset_index(drop = True)
for i in neprod.columns:
    s1 = pd.DataFrame()
    for j in range(len(ss['Регрессоры (тренды)'])):
        if i in ' '.join(ss['Регрессоры (тренды)'][j]): 
            s1 = s1.append(ss[ss.index == j])
    print(len(s1))